In [1]:
%load_ext autoreload
%autoreload 2
import whiskymetrics.database as db
from sqlalchemy.sql import select



Create data frame of review text

In [2]:
import pandas as pd

review = db.create_metadata().tables['review']
conn = db.get_connection()

s = select([review.c.id, review.c.link])
print(s)

url_df = pd.read_sql(sql=s,con=conn)
my_links = url_df
my_links.head()

SELECT review.id, review.link 
FROM review


,id,link
0,1,http://www.reddit.com/r/Scotch/comments/17e9n9...
1,2,https://www.reddit.com/r/Scotch/comments/43f6t...
2,3,http://www.reddit.com/r/Scotch/comments/2uddr0...
3,4,http://www.reddit.com/r/Scotch/comments/185jrq...
4,5,http://www.reddit.com/r/Scotch/comments/1a6bnw...


Get review text using PRAW

In [3]:
import praw
from numpy import nan

reddit = praw.Reddit(client_secret='vPPOuaLhteQcDgMUv0v5Qm0p_lM',user_agent='reviewbot (dev) v0.0.1 (by /u/TheJollyReaver)',client_id='4kmOMwhAe_AjMA')

fullnames = []
for link in my_links['link']:
    try:
        fullnames.append(reddit.submission(url = link).comments[0].fullname)
    except:
        fullnames.append(nan) 
    
my_links = my_links.assign(fullname=fullnames).dropna()
my_links.head()

,id,link,fullname
0,1,http://www.reddit.com/r/Scotch/comments/17e9n9...,t1_c84olxe
1,2,https://www.reddit.com/r/Scotch/comments/43f6t...,t1_czhr8am
2,3,http://www.reddit.com/r/Scotch/comments/2uddr0...,t1_co7d81e
3,4,http://www.reddit.com/r/Scotch/comments/185jrq...,t1_c8bsjx1
4,5,http://www.reddit.com/r/Scotch/comments/1a6bnw...,t1_c8uh0wg


In [4]:
text = [m.body for m in reddit.info(list(my_links['fullname']))]
my_links = my_links.assign(review_text=text).drop('link',1)
my_links.head()

,id,fullname,review_text
0,1,t1_c84olxe,"Dewar's. Highlands. **40%, $40.** \n\n**Nose:..."
1,2,t1_czhr8am,so last night a bunch of guys from my church g...
2,3,t1_co7d81e,Just back from an overnight outing with the ki...
3,4,t1_c8bsjx1,"Aberfeldy 12. Not bad, not fantastic, pretty c..."
4,5,t1_c8uh0wg,Just running though whats left of my swaps. No...


In [11]:
my_links = my_links.rename(columns={'fullname':'reddit_fullname'})
my_links.head()

,id,reddit_fullname,review_text
0,1,t1_c84olxe,"Dewar's. Highlands. **40%, $40.** \n\n**Nose:..."
1,2,t1_czhr8am,so last night a bunch of guys from my church g...
2,3,t1_co7d81e,Just back from an overnight outing with the ki...
3,4,t1_c8bsjx1,"Aberfeldy 12. Not bad, not fantastic, pretty c..."
4,5,t1_c8uh0wg,Just running though whats left of my swaps. No...


In [12]:
from whiskymetrics.etl import load
tbl = db.create_metadata().tables['review_text']
load(tbl,my_links)